In [1]:
import numpy as np
import pandas as pd
import sklearn
import pickle
import warnings

print("numpy: ", np.__version__)
print("pandas: ", pd.__version__)
print("sklearn: ", sklearn.__version__)

numpy:  1.26.4
pandas:  2.2.2
sklearn:  1.5.1


In [4]:
Luna_label = pd.read_excel('../../../data/Luna_training_labels_v2.xlsx')
Luna_data = pd.read_csv('../../../data/Luna_training_ULT.csv', encoding='utf-16')

In [11]:
import importlib
import preprocessing

importlib.reload(preprocessing)

<module 'preprocessing' from '/Users/snowy/wafers/Luna/Preprocessing/preprocessing.py'>

In [5]:
luna_cleaned = preprocessing.retrieve_test_data(Luna_data, Luna_label)
luna_cleaned

,0,1,2,3,4,5,6,7,8,9,...,3382,3383,3384,3385,3386,3387,3388,3389,3390,3391
Name,p_openXbegin_PAD_x_500uA__VBATPWR,p_openXbegin_PAD_x_500uA__VDDIO,p_openXbegin_PAD_x_500uA__NFC_GPIO0,p_openXbegin_PAD_x_500uA__NFC_GPIO1,p_openXbegin_PAD_x_500uA__NFC_GPIO2_AO,p_openXbegin_PAD_x_500uA__NFC_GPIO3_AO,p_openXbegin_PAD_x_500uA__NFC_GPIO4_AO,p_openXbegin_PAD_x_500uA__NFC_GPIO5,p_openXbegin_PAD_x_500uA__NFC_GPIO6,p_openXbegin_PAD_x_500uA__NFC_I2C_SCL,...,f_nvmXBistcomNvm1_x_x_vhsform__nvm1_FUNC (2),f_nvmXWritePatall0IfrNvm1_x_x_vhsform__nvm1_FU...,f_nvmXResetSocdataNvm1_x_x_vhsform__nvm1_FUNC,f_nvmXBulkSwitchingNvm1Nom_x_x_vnvn__nvm1_FUNC...,f_nvmXWritePatvformRdtyperdNvm1_x_x_vhsform__n...,f_nvmXWritePatvformRdtyperdNvm1_x_x_vhsform__r...,f_nvmXWritePatvformRdtyperdNvm1_x_x_vhsform__r...,f_nvmXWritePatvformRdtyperdRedenNvm1_x_x_vhsfo...,f_nvmXWritePatvformRdtyperdRedenNvm1_x_x_vhsfo...,f_nvmXWritePatvformRdtyperdRedenNvm1_x_x_vhsfo...
Distribution Type,outlier,longtail,outlier,outlier,outlier,outlier,outlier,outlier,outlier,outlier,...,functional,functional,functional,functional,functional,longtail,functional,functional,discrete,functional
1,-0.485749,-0.429951,-0.658435,-0.657163,-0.499695,-0.498676,-0.499989,-0.500702,-0.500391,-0.502783,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,-0.485348,-0.428958,-0.656091,-0.65626,-0.498872,-0.500664,-0.501008,-0.499002,-0.500574,-0.503204,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,-0.485128,-0.430099,-0.657087,-0.654594,-0.501647,-0.501958,-0.502802,-0.498472,-0.500946,-0.502849,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10740,-0.495285,-0.447545,-0.672854,-0.672481,-0.514087,-0.512276,-0.513175,-0.511027,-0.51001,-0.516845,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
10741,-0.496782,-0.450976,-0.673171,-0.675369,-0.51652,-0.518044,-0.519712,-0.512652,-0.51675,-0.521543,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
10742,-0.496654,-0.450121,-0.675772,-0.674746,-0.518838,-0.519225,-0.518069,-0.516434,-0.517565,-0.519458,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
10743,-0.495755,-0.449893,-0.675584,-0.675557,-0.516636,-0.517792,-0.518205,-0.517517,-0.519205,-0.521373,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [8]:
distribution_counts = luna_cleaned.T['Distribution Type'].value_counts()
distribution_counts

Distribution Type
functional    2099
outlier        649
longtail       299
discrete       225
normal          90
Untagged        16
bimodal          8
na               6
Name: count, dtype: int64

## Found luna dataset to have labels marked as 'Untagged' and 'na'

In [42]:
mask = luna_cleaned.loc['Distribution Type'].isin(['Untagged', 'na'])
luna_cleaned = luna_cleaned.loc[:, ~mask] # inverts the mask and filter out columns with those values in Distribution Type row
luna_cleaned

,0,1,2,3,4,5,6,7,8,9,...,3382,3383,3384,3385,3386,3387,3388,3389,3390,3391
Name,p_openXbegin_PAD_x_500uA__VBATPWR,p_openXbegin_PAD_x_500uA__VDDIO,p_openXbegin_PAD_x_500uA__NFC_GPIO0,p_openXbegin_PAD_x_500uA__NFC_GPIO1,p_openXbegin_PAD_x_500uA__NFC_GPIO2_AO,p_openXbegin_PAD_x_500uA__NFC_GPIO3_AO,p_openXbegin_PAD_x_500uA__NFC_GPIO4_AO,p_openXbegin_PAD_x_500uA__NFC_GPIO5,p_openXbegin_PAD_x_500uA__NFC_GPIO6,p_openXbegin_PAD_x_500uA__NFC_I2C_SCL,...,f_nvmXBistcomNvm1_x_x_vhsform__nvm1_FUNC (2),f_nvmXWritePatall0IfrNvm1_x_x_vhsform__nvm1_FU...,f_nvmXResetSocdataNvm1_x_x_vhsform__nvm1_FUNC,f_nvmXBulkSwitchingNvm1Nom_x_x_vnvn__nvm1_FUNC...,f_nvmXWritePatvformRdtyperdNvm1_x_x_vhsform__n...,f_nvmXWritePatvformRdtyperdNvm1_x_x_vhsform__r...,f_nvmXWritePatvformRdtyperdNvm1_x_x_vhsform__r...,f_nvmXWritePatvformRdtyperdRedenNvm1_x_x_vhsfo...,f_nvmXWritePatvformRdtyperdRedenNvm1_x_x_vhsfo...,f_nvmXWritePatvformRdtyperdRedenNvm1_x_x_vhsfo...
Distribution Type,outlier,longtail,outlier,outlier,outlier,outlier,outlier,outlier,outlier,outlier,...,functional,functional,functional,functional,functional,longtail,functional,functional,discrete,functional
1,-0.485749,-0.429951,-0.658435,-0.657163,-0.499695,-0.498676,-0.499989,-0.500702,-0.500391,-0.502783,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
2,-0.485348,-0.428958,-0.656091,-0.65626,-0.498872,-0.500664,-0.501008,-0.499002,-0.500574,-0.503204,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,-0.485128,-0.430099,-0.657087,-0.654594,-0.501647,-0.501958,-0.502802,-0.498472,-0.500946,-0.502849,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10740,-0.495285,-0.447545,-0.672854,-0.672481,-0.514087,-0.512276,-0.513175,-0.511027,-0.51001,-0.516845,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
10741,-0.496782,-0.450976,-0.673171,-0.675369,-0.51652,-0.518044,-0.519712,-0.512652,-0.51675,-0.521543,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
10742,-0.496654,-0.450121,-0.675772,-0.674746,-0.518838,-0.519225,-0.518069,-0.516434,-0.517565,-0.519458,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
10743,-0.495755,-0.449893,-0.675584,-0.675557,-0.516636,-0.517792,-0.518205,-0.517517,-0.519205,-0.521373,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0


In [43]:
data = luna_cleaned

with open('luna_values_dataframe.pkl', 'wb') as file:
    pickle.dump(data, file)

In [44]:
luna_features = preprocessing.retrieve_features(luna_cleaned)
luna_features

,Name,Distribution Type,Count,Unique_Count,Mean,Median,Std_Dev,IQR,Skewness,Kurtosis,...,Outliers_Zscore_prop,Outliers_IQR,Outliers_IQR_prop,Outliers_Tukey,Outliers_Tukey_prop,QQ Count,KS_Stat_norm,KS_P_value_norm,Shapiro_Stat,Shapiro_P_value
0,p_openXbegin_PAD_x_500uA__VBATPWR,outlier,10744.0,7003.0,0.109915,0.112241,0.035912,0.050512,1.605880,41.208402,...,0.000279,10744.0,1.0,2.0,0.000186,28.0,0.030411,4.572208e-09,0.930422,6.981834e-57
1,p_openXbegin_PAD_x_500uA__VDDIO,longtail,10744.0,7498.0,0.574196,0.593378,0.194096,0.281113,-0.321207,-0.532831,...,0.000000,0.0,0.0,0.0,0.000000,338.0,0.041946,7.297659e-17,0.983791,3.541060e-33
2,p_openXbegin_PAD_x_500uA__NFC_GPIO0,outlier,10744.0,7943.0,0.883541,0.886074,0.044407,0.057402,-1.193837,16.598391,...,0.001582,10744.0,1.0,7.0,0.000652,70.0,0.024858,3.362333e-06,0.958828,1.386030e-47
3,p_openXbegin_PAD_x_500uA__NFC_GPIO1,outlier,10744.0,7811.0,0.535529,0.542743,0.138557,0.186456,-0.200685,-0.340537,...,0.000465,10744.0,1.0,0.0,0.000000,362.0,0.024241,6.453124e-06,0.993951,7.075407e-21
4,p_openXbegin_PAD_x_500uA__NFC_GPIO2_AO,outlier,10744.0,7944.0,0.745949,0.747806,0.042186,0.058650,-0.816421,10.210895,...,0.000838,10744.0,1.0,5.0,0.000465,94.0,0.024024,8.079906e-06,0.968933,6.009823e-43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1997,NaN,NaN,10743.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,1.000000,0.000000e+00,0.000000,0.000000e+00
2778,NaN,NaN,10743.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,1.000000,0.000000e+00,0.000000,0.000000e+00
2779,NaN,NaN,10743.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,1.000000,0.000000e+00,0.000000,0.000000e+00
2780,NaN,NaN,10743.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.000000,0.0,1.000000,0.000000e+00,0.000000,0.000000e+00


In [13]:
luna_labeller = preprocessing.relabel_luna
luna_full = preprocessing.labelling_features(luna_features, luna_labeller)
luna_full

,Name,Target,Target_Encoded,Distribution Type,Count,Unique_Count,Mean,Median,Std_Dev,IQR,...,Outliers_Zscore_prop,Outliers_IQR,Outliers_IQR_prop,Outliers_Tukey,Outliers_Tukey_prop,QQ Count,KS_Stat_norm,KS_P_value_norm,Shapiro_Stat,Shapiro_P_value
0,p_openXbegin_PAD_x_500uA__VBATPWR,Outlier/Longtail,1,outlier,10744,7003,0.109915,0.112241,0.035912,0.050512,...,0.000279,10744,1.000000,2,0.000186,28,0.030411,4.572208e-09,0.930422,6.981834e-57
1,p_openXbegin_PAD_x_500uA__VDDIO,Outlier/Longtail,1,longtail,10744,7498,0.574196,0.593378,0.194096,0.281113,...,0.000000,0,0.000000,0,0.000000,338,0.041946,7.297659e-17,0.983791,3.541060e-33
2,p_openXbegin_PAD_x_500uA__NFC_GPIO0,Outlier/Longtail,1,outlier,10744,7943,0.883541,0.886074,0.044407,0.057402,...,0.001582,10744,1.000000,7,0.000652,70,0.024858,3.362333e-06,0.958828,1.386030e-47
3,p_openXbegin_PAD_x_500uA__NFC_GPIO1,Outlier/Longtail,1,outlier,10744,7811,0.535529,0.542743,0.138557,0.186456,...,0.000465,10744,1.000000,0,0.000000,362,0.024241,6.453124e-06,0.993951,7.075407e-21
4,p_openXbegin_PAD_x_500uA__NFC_GPIO2_AO,Outlier/Longtail,1,outlier,10744,7944,0.745949,0.747806,0.042186,0.058650,...,0.000838,10744,1.000000,5,0.000465,94,0.024024,8.079906e-06,0.968933,6.009823e-43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3387,f_nvmXWritePatvformRdtyperdNvm1_x_x_vhsform__r...,Outlier/Longtail,1,longtail,10743,11,0.000771,0.000000,0.018579,0.000000,...,0.004840,52,0.004840,52,0.004840,104,0.511717,0.000000e+00,0.018259,7.440625e-117
3388,f_nvmXWritePatvformRdtyperdNvm1_x_x_vhsform__r...,Other,0,functional,10743,1,0.000000,0.000000,0.000000,0.000000,...,0.000000,0,0.000000,0,0.000000,0,1.000000,0.000000e+00,0.000000,0.000000e+00
3389,f_nvmXWritePatvformRdtyperdRedenNvm1_x_x_vhsfo...,Other,0,functional,10743,1,0.000000,0.000000,0.000000,0.000000,...,0.000000,0,0.000000,0,0.000000,0,1.000000,0.000000e+00,0.000000,0.000000e+00
3390,f_nvmXWritePatvformRdtyperdRedenNvm1_x_x_vhsfo...,Other,0,discrete,10743,3,0.000186,0.000000,0.011815,0.000000,...,0.000279,3,0.000279,3,0.000279,6,0.506007,0.000000e+00,0.003628,2.985696e-117


In [14]:
binary_distribution_counts = luna_full['Target'].value_counts()
binary_distribution_counts

Target
Other               2444
Outlier/Longtail     948
Name: count, dtype: int64

In [15]:
luna_dropped = preprocessing.drop_null_tests(luna_full) # drop tests with any null features
luna_dropped

Number of rows dropped:  15


,Name,Target,Target_Encoded,Distribution Type,Count,Unique_Count,Mean,Median,Std_Dev,IQR,...,Outliers_Zscore_prop,Outliers_IQR,Outliers_IQR_prop,Outliers_Tukey,Outliers_Tukey_prop,QQ Count,KS_Stat_norm,KS_P_value_norm,Shapiro_Stat,Shapiro_P_value
0,p_openXbegin_PAD_x_500uA__VBATPWR,Outlier/Longtail,1,outlier,10744,7003,0.109915,0.112241,0.035912,0.050512,...,0.000279,10744,1.000000,2,0.000186,28,0.030411,4.572208e-09,0.930422,6.981834e-57
1,p_openXbegin_PAD_x_500uA__VDDIO,Outlier/Longtail,1,longtail,10744,7498,0.574196,0.593378,0.194096,0.281113,...,0.000000,0,0.000000,0,0.000000,338,0.041946,7.297659e-17,0.983791,3.541060e-33
2,p_openXbegin_PAD_x_500uA__NFC_GPIO0,Outlier/Longtail,1,outlier,10744,7943,0.883541,0.886074,0.044407,0.057402,...,0.001582,10744,1.000000,7,0.000652,70,0.024858,3.362333e-06,0.958828,1.386030e-47
3,p_openXbegin_PAD_x_500uA__NFC_GPIO1,Outlier/Longtail,1,outlier,10744,7811,0.535529,0.542743,0.138557,0.186456,...,0.000465,10744,1.000000,0,0.000000,362,0.024241,6.453124e-06,0.993951,7.075407e-21
4,p_openXbegin_PAD_x_500uA__NFC_GPIO2_AO,Outlier/Longtail,1,outlier,10744,7944,0.745949,0.747806,0.042186,0.058650,...,0.000838,10744,1.000000,5,0.000465,94,0.024024,8.079906e-06,0.968933,6.009823e-43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3387,f_nvmXWritePatvformRdtyperdNvm1_x_x_vhsform__r...,Outlier/Longtail,1,longtail,10743,11,0.000771,0.000000,0.018579,0.000000,...,0.004840,52,0.004840,52,0.004840,104,0.511717,0.000000e+00,0.018259,7.440625e-117
3388,f_nvmXWritePatvformRdtyperdNvm1_x_x_vhsform__r...,Other,0,functional,10743,1,0.000000,0.000000,0.000000,0.000000,...,0.000000,0,0.000000,0,0.000000,0,1.000000,0.000000e+00,0.000000,0.000000e+00
3389,f_nvmXWritePatvformRdtyperdRedenNvm1_x_x_vhsfo...,Other,0,functional,10743,1,0.000000,0.000000,0.000000,0.000000,...,0.000000,0,0.000000,0,0.000000,0,1.000000,0.000000e+00,0.000000,0.000000e+00
3390,f_nvmXWritePatvformRdtyperdRedenNvm1_x_x_vhsfo...,Other,0,discrete,10743,3,0.000186,0.000000,0.011815,0.000000,...,0.000279,3,0.000279,3,0.000279,6,0.506007,0.000000e+00,0.003628,2.985696e-117


In [ ]:
import pickle

data = {'luna_full': luna_full, 'luna_dropped': luna_dropped}

with open('luna_dataframes.pkl', 'wb') as file:
    pickle.dump(data, file)

# Testing pickled model and exensio function

In [23]:
import features

importlib.reload(features)

test_values = luna_cleaned[2:]
test_features = features.exensio_get_features(test_values)

with open('../luna_model.pkl', 'rb') as file:
    luna_model = pickle.load(file)

preds = luna_model.predict(test_features)